In [1]:
import pandas as pd #To hand with data 
import numpy as np #To math 
import seaborn as sns #to visualization
import matplotlib.pyplot as plt # to plot the graphs
import matplotlib.gridspec as gridspec # to do the grid of plots

from sklearn.metrics import classification_report
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import precision_score, recall_score, fbeta_score, confusion_matrix, precision_recall_curve, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path
import librosa
from sklearn.pipeline import Pipeline

#SMOTE
from imblearn.pipeline import make_pipeline # To do our transformation in a unique time
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced

#models
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier

In [2]:
first_wave = pd.read_csv(r'C:\Users\DELL\COV_Project\Files\Waves_Data\first_wave.csv')
second_wave = pd.read_csv(r'C:\Users\DELL\COV_Project\Files\Waves_Data\final_data_second_wave.csv')

### Encoding the Target Variable

In [3]:
first_wave['covid_status'] = first_wave['covid_status'].map({'healthy': 1, 'resp_illness_not_identified': 1, 'positive_asymp': 0, 'positive_mild': 0,
       'positive_moderate': 0, 'recovered_full': 1})

In [4]:
second_wave['covid_status'] = second_wave['covid_status'].map({'healthy': 1, 'resp_illness_not_identified': 1, 'positive_asymp': 0, 'positive_mild': 0,
       'positive_moderate': 0, 'recovered_full': 1})

#### Removing NaN values 

In [5]:
first_wave = first_wave.dropna()
second_wave = second_wave.dropna()

## Data Exploration

In [6]:
first_wave.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 451 entries, 0 to 483
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   covid_status        451 non-null    float64
 1   chroma_stft         451 non-null    float64
 2   rmse                451 non-null    float64
 3   spectral_centroid   451 non-null    float64
 4   spectral_bandwidth  451 non-null    float64
 5   rolloff             451 non-null    float64
 6   zero_crossing_rate  451 non-null    float64
 7   mfcc1               451 non-null    float64
 8   mfcc2               451 non-null    float64
 9   mfcc3               451 non-null    float64
 10  mfcc4               451 non-null    float64
 11  mfcc5               451 non-null    float64
 12  mfcc6               451 non-null    float64
 13  mfcc7               451 non-null    float64
 14  mfcc8               451 non-null    float64
 15  mfcc9               451 non-null    float64
 16  mfcc10  

In [7]:
second_wave.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497 entries, 0 to 555
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   covid_status        497 non-null    float64
 1   chroma_stft         497 non-null    float64
 2   rmse                497 non-null    float64
 3   spectral_centroid   497 non-null    float64
 4   spectral_bandwidth  497 non-null    float64
 5   rolloff             497 non-null    float64
 6   zero_crossing_rate  497 non-null    float64
 7   mfcc1               497 non-null    float64
 8   mfcc2               497 non-null    float64
 9   mfcc3               497 non-null    float64
 10  mfcc4               497 non-null    float64
 11  mfcc5               497 non-null    float64
 12  mfcc6               497 non-null    float64
 13  mfcc7               497 non-null    float64
 14  mfcc8               497 non-null    float64
 15  mfcc9               497 non-null    float64
 16  mfcc10  

In [8]:
# (1)https://www.analyticsvidhya.com/blog/2021/04/top-python-libraries-to-automate-exploratory-data-analysis-in-2021/
# import dtale
# import pandas as pd
# df = pd.read_csv(‘data.csv’)
# d = dtale.show(df)
# d.open_browser()

In [9]:
# #importing required packages
# import pandas as pd
# import pandas_profiling
# import numpy as np

# #descriptive statistics
# pandas_profiling.ProfileReport(first_wave)
# pandas_profiling.ProfileReport(second_wave)

In [10]:
second_wave.head()

,covid_status,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,1.0,0.530662,0.012492,3162.314178,3077.276596,5656.464720,0.051597,-539.624573,70.923248,-21.023085,...,-6.839964,-2.880063,0.642384,-6.130341,-3.196272,-3.263852,-6.482019,-0.151838,0.057630,-2.856626
1,1.0,0.291660,0.076092,1586.501846,1858.983904,2975.042230,0.036814,-340.324066,124.548439,9.032318,...,-22.191097,-1.023864,-3.656982,-14.091269,-8.621853,-5.527614,-6.184521,-4.964400,-7.813180,-11.550410
3,1.0,0.681028,0.065885,3255.085820,4549.176543,7710.263578,0.034163,-407.073547,61.919655,6.415006,...,5.473977,6.546779,8.448262,7.766901,2.511950,3.842897,1.211067,4.794546,2.072728,-1.464542
4,1.0,0.241772,0.020226,1408.958004,1503.902519,2554.820416,0.032323,-510.743195,50.490688,-16.352106,...,-3.257951,-2.551245,0.408354,-2.281638,-2.480655,-2.899497,-3.295894,-0.243877,-2.236765,-4.250204
5,0.0,0.511784,0.011890,3448.609718,4226.490094,6315.529337,0.061811,-517.450256,40.682167,-10.754708,...,-6.491390,-0.392513,0.345531,-3.572443,-0.002734,-2.758920,-5.120065,-0.831587,-2.985718,-3.265484


In [40]:
# (1) + https://www.kaggle.com/ahmettezcantekin/sweetviz-simple-and-quick-eda
# import sweetviz
# import pandas as pd
# #df = pd.read_csv('sample.csv')
# my_report  = sweetviz.analyze([second_wave,'Train'], target_feat='covid_status')
# my_report

# FIRST_WAVE = file:///C:/Users/DELL/COV_Project/Files/Waves_Data/first_wave_sweetviz.html
# SECOND_WAVE = file:///C:/Users/DELL/COV_Project/Files/Waves_Data/second_wave_sweetviz.html